In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU

In [ ]:
data = pd.read_csv("SARCASMMMIXEDD.csv",on_bad_lines='skip')
labels = data.label.values
sentences = data.comment.values
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,1,"I, for one, welcome this - it could lead to a ...",InitiumNovum,Anarcho_Capitalism,-1.0,-1.0,0.0,2015-09,9/22/2015 2:02,Ex-hedge funder buys rights to AIDS drug and r...
1,0,This is true,kyo_nimous_,Colts,1.0,1.0,0.0,2015-09,9/22/2015 2:05,You mean drafting and buying a bunch of WR's a...
2,1,Yeah they still believed in bad humours and bl...,SonofSonofSpock,aww,1.0,1.0,0.0,2015-09,9/22/2015 3:17,ahhh 1982. the year of all the best sources of...
3,1,R9 390 has 8gb vram therefore it must be even ...,iandj1,Amd,2.0,2.0,0.0,2015-09,9/22/2015 3:35,6GB on 980Ti to 4GB on Fury reverses AMD's adv...
4,1,Probably should've put a,Johnie4usc,nba,-2.0,-2.0,0.0,2015-09,9/22/2015 3:58,"yeah, not really. that was highlighted all ove..."


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732 entries, 0 to 1731
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           1732 non-null   int64  
 1   comment         1732 non-null   object 
 2   author          865 non-null    object 
 3   subreddit       865 non-null    object 
 4   score           865 non-null    float64
 5   ups             865 non-null    float64
 6   downs           865 non-null    float64
 7   date            865 non-null    object 
 8   created_utc     865 non-null    object 
 9   parent_comment  865 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 135.4+ KB


In [ ]:
#checking for null values in train data
data.isnull().sum()

label               0
comment             0
author            867
subreddit         867
score             867
ups               867
downs             867
date              867
created_utc       867
parent_comment    867
dtype: int64

In [ ]:
deletecolumn = ['author','subreddit','score','ups','downs','date','created_utc','parent_comment']
data.drop(columns=deletecolumn, inplace=True)

In [ ]:
data.head(10)

,label,comment
0,1,"I, for one, welcome this - it could lead to a ..."
1,0,This is true
2,1,Yeah they still believed in bad humours and bl...
3,1,R9 390 has 8gb vram therefore it must be even ...
4,1,Probably should've put a
5,1,People can't understand sarcasm without a
6,1,Did a great job not messaging you again
7,1,Or you can just down-vote it
8,1,Because love has no place in hip hop
9,1,#occupyPLEX2016 We are the 99%!


In [ ]:
data.describe(include='object')

,comment
count,1732
unique,1731
top,You forgot
freq,2


In [ ]:
data['comment'].duplicated().sum()

1

In [ ]:
data=data.drop(data[data['comment'].duplicated()].index,axis=0)

In [ ]:
import re
import string
from nltk.corpus import stopwords

# Initialize stopwords and punctuation
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Removing the stopwords from text
def split_into_words(text):
    # Split into words by white space
    words = text.split()
    return words

def to_lower_case(words):
    # Convert to lower case
    words = [word.lower() for word in words]
    return words

def remove_punctuation(words):
    # Prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # Remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def keep_alphabetic(words):
    # Remove remaining tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
    return words

def remove_stopwords(words):
    # Filter out stop words
    words = [w for w in words if not w in stop]
    return words

def remove_hashtags(words):
    # Remove words that start with #
    words = [word for word in words if not word.startswith('#')]
    return words

def to_sentence(words):
    # Join words to a sentence
    return ' '.join(words)

# Removing the noisy text
def denoise_text(text):
    words = split_into_words(text)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    words = keep_alphabetic(words)
    words = remove_stopwords(words)
    words = remove_hashtags(words)  # Remove hashtags
    return to_sentence(words)

# Example usage
example_text = "This is an example text with a #hashtag and some punctuation!"
cleaned_text = denoise_text(example_text)
print("Cleaned Text:", cleaned_text)


Cleaned Text: example text hashtag punctuation


In [ ]:
data['comment'] = data['comment'].astype(str) #deleting duplicate

In [ ]:
#Apply function on review column
data['comment']=data['comment'].apply(denoise_text)

In [ ]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Define the pretrained model name
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=True)

# Define the encoder function
def encoder(sentences):
    ids = []
    for sentence in sentences:
        if isinstance(sentence, str):  # Ensure the sentence is a string
            encoding = tokenizer.encode_plus(
                sentence,
                max_length=16,
                truncation=True,
                add_special_tokens=True,
                return_token_type_ids=False,
                pad_to_max_length=True,
                return_attention_mask=False
            )
            ids.append(encoding['input_ids'])
        else:
            raise ValueError(f"Expected string but got {type(sentence)}: {sentence}")
    return ids

# Filter out NaN values from sentences and labels
filtered_sentences = [s for s in sentences if isinstance(s, str)]
filtered_labels = [labels[i] for i in range(len(labels)) if isinstance(sentences[i], str)]

# Train test split
train_sents, test_sents, train_labels, test_labels = train_test_split(filtered_sentences, filtered_labels, test_size=0.20)

# Encode the train and test sentences
train_ids = encoder(train_sents)
test_ids = encoder(test_sents)

print("Train IDs:", train_ids)
print("Test IDs:", test_ids)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Train IDs: [[101, 1045, 2089, 2298, 3407, 2006, 1996, 2648, 2978, 2006, 1996, 2503, 1045, 1005, 1049, 102], [101, 2515, 3087, 2025, 2113, 3159, 4302, 2288, 5117, 1029, 1045, 24260, 30102, 30108, 2102, 102], [101, 11928, 5332, 15775, 10559, 15030, 11113, 17710, 5292, 4213, 28774, 2063, 7890, 2121, 24182, 102], [101, 7151, 11921, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 14926, 2386, 4063, 2075, 2663, 1013, 3279, 2636, 102, 0, 0, 0, 0, 0, 0], [101, 1030, 3590, 24281, 3406, 14754, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2293, 2009, 2043, 7144, 1010, 4297, 5644, 18688, 3686, 4257, 15416, 2272, 2067, 102], [101, 2066, 2273, 2224, 6625, 7415, 3111, 2000, 5342, 2256, 12626, 2100, 1029, 102, 0, 0], [101, 1030, 9915, 2527, 2620, 2581, 2278, 1030, 10958, 7507, 9050, 22345, 2100, 1030, 14673, 102], [101, 2823, 1045, 2514, 2066, 2035, 2023, 3947, 2003, 2074, 2066, 13423, 6323, 4372, 26775, 102], [101, 2009, 1005, 1055, 2042, 1037, 13318, 2197, 2154, 1012, 4098, 2098, 2041, 2035, 2026

In [ ]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.Model(inputs=[input_word_ids], outputs=output)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
for layer in model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer  [(None, 16)]              0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPo   109482240 
 )                           olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 16, 768),                        
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                   

In [ ]:
history = model.fit(x = train_ids, y = train_labels, epochs = 5, verbose = 1, batch_size = 32, validation_data = (test_ids, test_labels))

Epoch 1/5
44/44 [==============================] - 186s 3s/step - loss: 0.7014 - accuracy: 0.5177 - val_loss: 0.6907 - val_accuracy: 0.5360
Epoch 2/5
 5/44 [==>...........................] - ETA: 1:21 - loss: 0.7239 - accuracy: 0.5188

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(model, string):
  plt.plot(model.history[string])
  plt.plot(model.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend(['Training ' +string, 'Validation '+string])
  plt.show()

# Plot the accuracy and loss
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: name 'history' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predictions
predictions = (model.predict(test_ids) > 0.5).astype("int32")

# Accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

# Precision
precision = precision_score(test_labels, predictions)
print("Precision:", precision)

# Recall
recall = recall_score(test_labels, predictions)
print("Recall:", recall)

# F1 Score
f1 = f1_score(test_labels, predictions)
print("F1 Score:", f1)


11/11 [==============================] - 32s 3s/step
Accuracy: 0.5360230547550432
Precision: 0.8171428571428572
Recall: 0.5257352941176471
F1 Score: 0.639821029082774
